In [1]:
import numpy as np
import pygame
import sys
import math

BLUE = (0, 0, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
YELLOW = (255, 255, 0)

ROWS = 6
COLS = 7
SQUARESIZE = 100
RADIUS = int(SQUARESIZE/2 - 5)

width = COLS * SQUARESIZE
height = (ROWS + 1) * SQUARESIZE

size = (width, height)

PLAYER = 1
AI = 2
EMPTY = 0

pygame.mixer.init()

# Load sound effects
drop_sound = pygame.mixer.Sound("coin-drop.mp3")
win_sound = pygame.mixer.Sound("woo-hoo.mp3")

pygame 2.6.0 (SDL 2.28.4, Python 3.9.17)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
def create_board():
    return np.zeros((ROWS, COLS))

def drop_piece(board, row, col, piece):
    board[row][col] = piece

def is_valid_location(board, col):
    return board[ROWS-1][col] == 0

def get_next_open_row(board, col):
    for r in range(ROWS):
        if board[r][col] == 0:
            return r

In [3]:
def winning_move(board, piece):
    # Check horizontal locations
    for c in range(COLS-3):
        for r in range(ROWS):
            if board[r][c] == piece and board[r][c+1] == piece and board[r][c+2] == piece and board[r][c+3] == piece:
                return True

    # Check vertical locations
    for c in range(COLS):
        for r in range(ROWS-3):
            if board[r][c] == piece and board[r+1][c] == piece and board[r+2][c] == piece and board[r+3][c] == piece:
                return True

    # Check positively sloped diagonals
    for c in range(COLS-3):
        for r in range(ROWS-3):
            if board[r][c] == piece and board[r+1][c+1] == piece and board[r+2][c+2] == piece and board[r+3][c+3] == piece:
                return True

    # Check negatively sloped diagonals
    for c in range(COLS-3):
        for r in range(3, ROWS):
            if board[r][c] == piece and board[r-1][c+1] == piece and board[r-2][c+2] == piece and board[r-3][c+3] == piece:
                return True

    return False

In [4]:
def draw_board(board):
    for c in range(COLS):
        for r in range(ROWS):
            pygame.draw.rect(screen, BLUE, (c*SQUARESIZE, r*SQUARESIZE+SQUARESIZE, SQUARESIZE, SQUARESIZE))
            pygame.draw.circle(screen, BLACK, (int(c*SQUARESIZE+SQUARESIZE/2), int(r*SQUARESIZE+SQUARESIZE+SQUARESIZE/2)), RADIUS)
    
    for c in range(COLS):
        for r in range(ROWS):		
            if board[r][c] == PLAYER:
                pygame.draw.circle(screen, RED, (int(c*SQUARESIZE+SQUARESIZE/2), height-int(r*SQUARESIZE+SQUARESIZE/2)), RADIUS)
            elif board[r][c] == AI: 
                pygame.draw.circle(screen, YELLOW, (int(c*SQUARESIZE+SQUARESIZE/2), height-int(r*SQUARESIZE+SQUARESIZE/2)), RADIUS)
    pygame.display.update()

In [5]:
def evaluate_window(window, piece):
    score = 0
    opp_piece = PLAYER if piece == AI else AI

    if window.count(piece) == 4:
        score += 100
    elif window.count(piece) == 3 and window.count(EMPTY) == 1:
        score += 5
    elif window.count(piece) == 2 and window.count(EMPTY) == 2:
        score += 2

    if window.count(opp_piece) == 3 and window.count(EMPTY) == 1:
        score -= 4

    return score

In [6]:
def score_position(board, piece):
    score = 0

    # Score center column
    center_array = [int(i) for i in list(board[:, COLS//2])]
    center_count = center_array.count(piece)
    score += center_count * 3

    # Score Horizontal
    for r in range(ROWS):
        row_array = [int(i) for i in list(board[r,:])]
        for c in range(COLS-3):
            window = row_array[c:c+4]
            score += evaluate_window(window, piece)

    # Score Vertical
    for c in range(COLS):
        col_array = [int(i) for i in list(board[:,c])]
        for r in range(ROWS-3):
            window = col_array[r:r+4]
            score += evaluate_window(window, piece)

    # Score positive sloped diagonal
    for r in range(ROWS-3):
        for c in range(COLS-3):
            window = [board[r+i][c+i] for i in range(4)]
            score += evaluate_window(window, piece)

    # Score negative sloped diagonal
    for r in range(ROWS-3):
        for c in range(COLS-3):
            window = [board[r+3-i][c+i] for i in range(4)]
            score += evaluate_window(window, piece)

    return score

In [7]:
def is_terminal_node(board):
    return winning_move(board, PLAYER) or winning_move(board, AI) or len(get_valid_locations(board)) == 0

def get_valid_locations(board):
    return [col for col in range(COLS) if is_valid_location(board, col)]

In [8]:
def minimax(board, depth, alpha, beta, maximizing_player):
    valid_locations = get_valid_locations(board)
    is_terminal = is_terminal_node(board)
    
    if depth == 0 or is_terminal:
        if is_terminal:
            if winning_move(board, AI):
                return (None, 100000000000000)
            elif winning_move(board, PLAYER):
                return (None, -10000000000000)
            else:  # Game is over, no more valid moves
                return (None, 0)
        else:  # Depth is zero
            return (None, score_position(board, AI))
    
    if maximizing_player:
        value = -np.inf
        column = np.random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            drop_piece(b_copy, row, col, AI)
            new_score = minimax(b_copy, depth-1, alpha, beta, False)[1]
            if new_score > value:
                value = new_score
                column = col
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return column, value

    else:  # Minimizing player
        value = np.inf
        column = np.random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            drop_piece(b_copy, row, col, PLAYER)
            new_score = minimax(b_copy, depth-1, alpha, beta, True)[1]
            if new_score < value:
                value = new_score
                column = col
            beta = min(beta, value)
            if alpha >= beta:
                break
        return column, value

In [9]:
def get_winning_pieces(board, piece):
    # Check horizontal locations
    for c in range(COLS-3):
        for r in range(ROWS):
            if board[r][c] == piece and board[r][c+1] == piece and board[r][c+2] == piece and board[r][c+3] == piece:
                return [(r,c), (r,c+1), (r,c+2), (r,c+3)]

    # Check vertical locations
    for c in range(COLS):
        for r in range(ROWS-3):
            if board[r][c] == piece and board[r+1][c] == piece and board[r+2][c] == piece and board[r+3][c] == piece:
                return [(r,c), (r+1,c), (r+2,c), (r+3,c)]

    # Check positively sloped diagonals
    for c in range(COLS-3):
        for r in range(ROWS-3):
            if board[r][c] == piece and board[r+1][c+1] == piece and board[r+2][c+2] == piece and board[r+3][c+3] == piece:
                return [(r,c), (r+1,c+1), (r+2,c+2), (r+3,c+3)]

    # Check negatively sloped diagonals
    for c in range(COLS-3):
        for r in range(3, ROWS):
            if board[r][c] == piece and board[r-1][c+1] == piece and board[r-2][c+2] == piece and board[r-3][c+3] == piece:
                return [(r,c), (r-1,c+1), (r-2,c+2), (r-3,c+3)]

    return []

In [10]:
def create_button(x, y, width, height, text, color, text_color):
    button = pygame.Rect(x, y, width, height)
    pygame.draw.rect(screen, color, button)
    font = pygame.font.SysFont("monospace", 20)
    text_surf = font.render(text, True, text_color)
    text_rect = text_surf.get_rect(center=button.center)
    screen.blit(text_surf, text_rect)
    return button, text_surf

def reset_game():
    return create_board(), 0, False

In [11]:
def play_game():
    board, turn, game_over = reset_game()
    
    draw_board(board)
    pygame.display.update()

    # Create Reset button
    reset_button, _ = create_button(width - 140, 10, 60, 30, "Reset", (255, 165, 0), BLACK)  # Orange color

    # Create Close Game button
    close_game_button, _ = create_button(width - 70, 10, 60, 30, "Close", (255, 0, 0), BLACK)  # Red color

    while True:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                sys.exit()

            if event.type == pygame.MOUSEMOTION and not game_over:
                pygame.draw.rect(screen, BLACK, (0,0, width, SQUARESIZE))
                posx = event.pos[0]
                if turn == 0:
                    pygame.draw.circle(screen, RED, (posx, int(SQUARESIZE/2)), RADIUS)
                
                # Redraw Reset and Close Game buttons
                reset_button, _ = create_button(width - 140, 10, 60, 30, "Reset", (255, 165, 0), BLACK)
                close_game_button, _ = create_button(width - 70, 10, 60, 30, "Close", (255, 0, 0), BLACK)

            pygame.display.update()

            if event.type == pygame.MOUSEBUTTONDOWN:
                # Check if Reset button is clicked
                if reset_button.collidepoint(event.pos):
                    board, turn, game_over = reset_game()
                    draw_board(board)
                    pygame.display.update()
                    continue

                # Check if Close Game button is clicked
                if close_game_button.collidepoint(event.pos):
                    pygame.quit()
                    sys.exit()

                if game_over:
                    # Check if Play Again button is clicked
                    if play_again_button.collidepoint(event.pos):
                        board, turn, game_over = reset_game()
                        draw_board(board)
                        pygame.display.update()
                        continue

                if not game_over and turn == 0:
                    posx = event.pos[0]
                    col = int(math.floor(posx/SQUARESIZE))

                    if is_valid_location(board, col):
                        row = get_next_open_row(board, col)
                        drop_piece(board, row, col, PLAYER)

                        drop_sound.play()

                        if winning_move(board, PLAYER):
                            label = myfont.render("Player 1 wins!!", 1, RED)
                            screen.blit(label, (40,10))
                            win_sound.play()
                            game_over = True
                            winning_pieces = get_winning_pieces(board, PLAYER)

                        turn += 1
                        turn = turn % 2

                        draw_board(board)

        if not game_over and turn == 1:		
            col, minimax_score = minimax(board, 5, -math.inf, math.inf, True)

            if is_valid_location(board, col):
                pygame.time.wait(500)
                row = get_next_open_row(board, col)
                drop_piece(board, row, col, AI)

                drop_sound.play()

                if winning_move(board, AI):
                    label = myfont.render("Player 2 wins!!", 1, YELLOW)
                    screen.blit(label, (40,10))
                    win_sound.play()
                    game_over = True
                    winning_pieces = get_winning_pieces(board, AI)

                draw_board(board)

                turn += 1
                turn = turn % 2

        if game_over:
            # Winning animation
            for _ in range(5):  # Flash 5 times
                for piece in winning_pieces:
                    row, col = piece
                    pygame.draw.circle(screen, BLACK, (int(col*SQUARESIZE+SQUARESIZE/2), height-int(row*SQUARESIZE+SQUARESIZE/2)), RADIUS)
                pygame.display.update()
                pygame.time.wait(200)
                
                for piece in winning_pieces:
                    row, col = piece
                    color = RED if board[row][col] == PLAYER else YELLOW
                    pygame.draw.circle(screen, color, (int(col*SQUARESIZE+SQUARESIZE/2), height-int(row*SQUARESIZE+SQUARESIZE/2)), RADIUS)
                pygame.display.update()
                pygame.time.wait(200)

            # Create and draw Play Again button
            play_again_button, _ = create_button(width//2 - 50, height - 60, 100, 40, "Play Again", (0, 255, 0), BLACK)
            
            pygame.display.update()

        # Always redraw Reset and Close Game buttons
        reset_button, _ = create_button(width - 140, 10, 60, 30, "Reset", (255, 165, 0), BLACK)
        close_game_button, _ = create_button(width - 70, 10, 60, 30, "Close", (255, 0, 0), BLACK)
        pygame.display.update()

In [ ]:
pygame.init()
screen = pygame.display.set_mode(size)
myfont = pygame.font.SysFont("monospace", 75)

play_game()